<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/02_intermediate/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.enable_eager_execution()
tfk = tf.keras
tfe = tf.contrib.eager

In [32]:
# Hyper parameters
num_epochs = 25
num_classes = 10
batch_size = 512
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tfk.datasets.mnist.load_data()

print("training_data: ", x_train.shape)
print("test_data: ", x_test.shape)
print("training_label: ", y_train.shape)
print("test_label: ", y_test.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000,)
test_label:  (10000,)


In [33]:
x_train_eager = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test_eager = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_train_eager = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))
y_test_eager = tf.reshape(tf.one_hot(y_test, 10), (-1, 10))

print("training_data: ", x_train_eager.shape)
print("test_data: ", x_test_eager.shape)
print("training_label: ", y_train_eager.shape)
print("test_label: ", y_test_eager.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000, 10)
test_label:  (10000, 10)


### DataSet
You make Dataset using `tf.data.Dataset` Class but Keras API doesn't need this dataset. If you write training loop code manually, `Dataset` class is very useful. And using keras API, you need numpy.array inputs instead of tf.Tensor. I don't know why...so you only need numpy preprocessing (or get numpy.array from tf.Tensor using numpy() method after preprocessing using function of tf).


In [0]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train_eager, y_train_eager))
    .batch(batch_size)
    .shuffle(10000)
)
train_dataset = train_dataset.repeat()

In [0]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test_eager, y_test_eager))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = test_dataset.repeat()

In [0]:
class RNN(tfk.Model):
    def __init__(self, hidden_size=10, num_layers=2, num_classes=10):
        super(RNN, self).__init__(name='mnist_rnn')
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = self.get_lstm_layers(hidden_size, num_layers)            
        self.fc = tfk.layers.Dense(num_classes, activation="softmax")
    
    @staticmethod
    def get_lstm_layers(hidden_size, num_layers):
        lstm_layers = []
        for i in range(num_layers-1):
            lstm_layers.append(
                tfk.layers.CuDNNLSTM(units=hidden_size, return_sequences=True)
            )
        # the final layer return only final sequence
        # if you need all sequences, you have to write return_sequences=True.
        lstm_layers.append(tfk.layers.CuDNNLSTM(units=hidden_size))
        return tfk.Sequential(lstm_layers)
        
    def call(self, x):        
        # Forward propagate LSTM
        out = self.lstm(x)
        out = self.fc(out)
        return out

In [0]:
model = RNN()

In [38]:
optimizer = tf.train.AdamOptimizer(learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=["accuracy"])

# Eager Execution initialize parameters when using model.call()
model(x_train_eager[:50])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    multiple                  2480      
_________________________________________________________________
dense_3 (Dense)              multiple                  110       
Total params: 2,590
Trainable params: 2,590
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(x=x_train_eager.numpy(), 
          y=y_train_eager.numpy(), 
          validation_split=0.2, 
          epochs=num_epochs,
          batch_size=batch_size)

Epoch 1/25
94/94 [==============================] - 4s 47ms/step - loss: 2.1856 - acc: 0.2238 - val_loss: 1.9467 - val_acc: 0.3433
Epoch 2/25
94/94 [==============================] - 4s 45ms/step - loss: 1.6989 - acc: 0.4228 - val_loss: 1.4462 - val_acc: 0.5258
Epoch 3/25
94/94 [==============================] - 4s 46ms/step - loss: 1.2604 - acc: 0.6018 - val_loss: 1.0583 - val_acc: 0.6913
Epoch 4/25
94/94 [==============================] - 4s 46ms/step - loss: 0.9625 - acc: 0.7154 - val_loss: 0.8442 - val_acc: 0.7549
Epoch 5/25
94/94 [==============================] - 4s 45ms/step - loss: 0.8116 - acc: 0.7575 - val_loss: 0.7295 - val_acc: 0.7807
Epoch 6/25
94/94 [==============================] - 4s 45ms/step - loss: 0.7145 - acc: 0.7822 - val_loss: 0.6535 - val_acc: 0.8047
Epoch 7/25
94/94 [==============================] - 4s 45ms/step - loss: 0.6454 - acc: 0.8027 - val_loss: 0.5999 - val_acc: 0.8229
Epoch 8/25
94/94 [==============================] - 4s 45ms/step - loss: 0.5950 - a

In [41]:
test_loss, test_acc = model.evaluate(x=x_test_eager.numpy(), 
                                     y=y_test_eager.numpy())

print("test_accracy: ", test_acc)

313/313 [==============================] - 5s 17ms/step
test_accracy:  0.9114
